# Resume Screening with LLMs

**Goal**: Build a resume scoring system using structured outputs

## What You'll Learn
1. Load resume data from CSV
2. Use structured outputs to analyze resumes
3. Create a scoring system (0-100)

## Setup

In [ ]:
# Configuration
import json
from resume_utils import load_resumes, load_job_description, analyze_resume
import os

OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY", '')  # Paste your key here

if not OPENROUTER_API_KEY or OPENROUTER_API_KEY.strip() == "":
    raise RuntimeError(
        "⚠️  Please set OPENROUTER_API_KEY above before running this notebook.\n"
        "Get your key from: https://openrouter.ai/keys"
    )

print("✓ API key configured")


✓ API key configured


## Part 1: Loading Resume Data

In [16]:
# Load all resumes into a dictionary
resumes = load_resumes('../data/resumes_final.csv')

print(f"Loaded {len(resumes)} resumes")
print(f"\nFirst 5 resume IDs: {list(resumes.keys())[:5]}")

Loaded 130 resumes

First 5 resume IDs: ['10089434', '10247517', '10265057', '10553553', '10641230']


In [21]:
# View a single resume
sample_id = list(resumes.keys())[1]
sample_resume = resumes[sample_id]

print(f"Resume ID: {sample_resume['ID']}")
print(f"\nResume text (first 500 characters):")
print("="*70)
print(sample_resume['Resume_str'][:500])
print("...")

Resume ID: 10247517

Resume text (first 500 characters):
         INFORMATION TECHNOLOGY MANAGER       Professional Summary    Possesses an extensive background in Information Technology Management, along with a Masters of Science degree and multiple certifications.  Excels in planning, implementing, and evaluating the systems, infrastructure, and staffing necessary to execute complex initiatives and meet deadlines in dynamic, fast-paced environments; adept at overseeing and participating in the installation, configuration, maintenance, and upgrade of
...


In [17]:
# Load the job description
job_description = load_job_description('../data/job_req_junior.md')

print(f"✓ Job description loaded")
print(f"\nJob description (first 400 characters):")
print("="*70)
print(job_description[:400])
print("...")


✓ Job description loaded

Job description (first 400 characters):
# Junior Software Engineer - Python

**Location:** Hybrid (Chicago, IL or Remote)
**Department:** Engineering
**Employment Type:** Full-Time
**Experience Level:** Entry Level (0-2 years)

## About the Role

We are looking for a motivated Junior Software Engineer to join our growing team. This role is perfect for recent graduates, bootcamp alumni, or early-career developers who are passionate about
...


## Part 2: Structured Output with LLMs

The `analyze_resume()` function takes:
1. **api_key**: Your OpenRouter API key
2. **prompt**: What you want to analyze
3. **resume_text**: The resume to analyze
4. **output_schema**: The JSON structure you want back
5. **model**: Which LLM to use (optional)

And returns structured JSON output.

## Example 1: Extract Technical Skills

In [18]:
# Define what we want to extract
prompt = "Extract the technical skills, programming languages, frameworks, and technologies from this resume."

# Define the output structure
output_schema = """
{
  "programming_languages": ["list of languages"],
  "frameworks_libraries": ["list of frameworks"],
  "databases": ["list of databases"],
  "cloud_platforms": ["list of cloud platforms"],
  "tools": ["list of development tools"]
}
"""

# Analyze the resume: note that this requires an output schema
result = analyze_resume(
    OPENROUTER_API_KEY,
    prompt,
    sample_resume['Resume_str'],
    output_schema
)

if result['error']:
    print(f"❌ Error: {result['error']}")
else:
    print("✓ Skills extracted successfully\n")
    print(json.dumps(result['result'], indent=2))
    print(f"\nTokens used: {result['usage'].get('total_tokens', 0)}")

✓ Skills extracted successfully

{
  "programming_languages": [
    "PowerShell",
    "VBScript",
    "HTML5",
    "CSS3"
  ],
  "frameworks_libraries": [
    ".Net Framework 4/4.5",
    "MVC 4"
  ],
  "databases": [],
  "cloud_platforms": [
    "Microsoft Azure",
    "Office 365"
  ],
  "tools": [
    "Active Directory",
    "Group Policy Objects",
    "Microsoft Exchange",
    "VMWare",
    "StorSimple",
    "Twinstrata",
    "Team Foundation Server",
    "Visual Studio",
    "Cacti",
    "Hyperion"
  ]
}

Tokens used: 925


In [22]:
# Define what we want to extract - compare against actual job description
prompt = f"""Extract the technical skills, programming languages, frameworks, and technologies from this resume and compare them to the job description below. Then give a score from 0-100 on how well this candidate matches the job requirements.

JOB DESCRIPTION:
{job_description}
"""

# Define the output structure
output_schema = """
{
    "match_score": "integer from 0 to 100",
}
"""

# Analyze the resume: note that this requires an output schema
result = analyze_resume(
    OPENROUTER_API_KEY,
    prompt,
    sample_resume['Resume_str'],
    output_schema
)

if result['error']:
    print(f"❌ Error: {result['error']}")
else:
    print("✓ Skills extracted successfully\n")
    print(json.dumps(result['result'], indent=2))
    print(f"\nTokens used: {result['usage'].get('total_tokens', 0)}")


✓ Skills extracted successfully

{
  "match_score": 25
}

Tokens used: 2286


## Example 2: Batch Score Multiple Resumes

Analyze multiple resumes at once and get scores for all of them.


In [27]:
# Define how many resumes to process
num_resumes = 10  # Change this to process more or fewer resumes

# Get the resume IDs
resume_ids = list(resumes.keys())[:num_resumes]

# Store results
screening_results = []

print(f"📋 Screening {len(resume_ids)} resumes...\n")

# Process each resume
for idx, resume_id in enumerate(resume_ids, 1):
    resume_data = resumes[resume_id]
    
    # Create the comparison prompt
    prompt = f"""Extract the technical skills, programming languages, frameworks, and technologies from this resume and compare them to the job description below.
    Then give a score from on how well this candidate matches the job requirements.
    Every matching coding skill should increase the score by 5, while missing required skills should decrease it 5.
    Each year of relevant experience should add 2 points to the score.
    Also give me a quick summary of the candidate working expreince and skills.

JOB DESCRIPTION:
{job_description}
"""

    # Define output schema
    output_schema = """
{
    "match_score": "integer from 0 to 100",
    "summary": "brief summary of candidate's experience and skills"
  
}
"""

    # Analyze the resume
    result = analyze_resume(
        OPENROUTER_API_KEY,
        prompt,
        resume_data['Resume_str'],
        output_schema
    )

    # Store result with resume ID
    if not result['error']:
        screening_results.append({
            "resume_id": resume_id,
            "analysis": result['result'],
            "tokens": result['usage'].get('total_tokens', 0)
        })
        match_score = result['result'].get('match_score', 0)
        print(f"✓ [{idx}/{len(resume_ids)}] Resume {resume_id}: Score {match_score}")
    else:
        print(f"❌ [{idx}/{len(resume_ids)}] Resume {resume_id}: Error - {result['error']}")

print(f"\n✓ Screening complete! Processed {len(screening_results)} resumes")


📋 Screening 10 resumes...

✓ [1/10] Resume 10089434: Score 44
✓ [2/10] Resume 10247517: Score 45
✓ [3/10] Resume 10265057: Score 54
✓ [4/10] Resume 10553553: Score 34
✓ [5/10] Resume 10641230: Score 42
✓ [6/10] Resume 10839851: Score 10
✓ [7/10] Resume 10840430: Score 45
✓ [8/10] Resume 11580408: Score 42
✓ [9/10] Resume 11584809: Score 64
✓ [10/10] Resume 11957080: Score 24

✓ Screening complete! Processed 10 resumes


In [28]:
# Display results summary sorted by match score
print("\n📊 SCREENING RESULTS SUMMARY\n")
print("="*80)

# Sort by match score (descending)
sorted_results = sorted(
    screening_results, 
    key=lambda x: x['analysis'].get('match_score', 0),
    reverse=True
)

for rank, result in enumerate(sorted_results, 1):
    resume_id = result['resume_id']
    score = result['analysis'].get('match_score', 0)
    summary = result['analysis'].get('summary', 'No summary available')
    
    print(f"\n{rank}. Resume ID: {resume_id}")
    print(f"   Match Score: {score}/100")
    print(f"   Summary: {summary}")
    print("-"*80)

# Summary stats
avg_score = sum(r['analysis'].get('match_score', 0) for r in screening_results) / len(screening_results) if screening_results else 0
print(f"\n📈 AVERAGE MATCH SCORE: {avg_score:.1f}/100")
print(f"🏆 TOP CANDIDATE: {sorted_results[0]['resume_id'] if sorted_results else 'N/A'} ({sorted_results[0]['analysis'].get('match_score', 0)}/100)" if sorted_results else "No results")



📊 SCREENING RESULTS SUMMARY



TypeError: '<' not supported between instances of 'str' and 'int'